In [2]:
#import all packages
import pandas as pd
import numpy as np
import re as re
import nltk
import string, unicodedata
from nltk.stem.porter import *

from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from datetime import datetime, timedelta

In [5]:
def reduce_lengthening(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

In [6]:
# datafile
data_file = "AllData_Raw.csv"
data = pd.read_csv(data_file)
data.columns = data.columns.str.strip().str.replace(" ","_") # reason for doing this is cause cannot get columns with space into a list

#reformat the respective columns

#lower case
data['Review'] = [word.lower() for word in data['Review']]

#Strip off the Variation Word
data['Product_Purchase'] = data['Product_Purchase'].replace('Variation', '')

#Change Date of Review to DateTime
#data['Date_Of_Review'] = pd.to_datetime(data['Date_Of_Review'])

data.info()
data.head()
#data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9002 entries, 0 to 9001
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Platform          9002 non-null   object 
 1   Brand             9002 non-null   object 
 2   Category          9002 non-null   object 
 3   Product_Name      9002 non-null   object 
 4   Price             9002 non-null   float64
 5   Reviewer          8980 non-null   object 
 6   Review            9002 non-null   object 
 7   Product_Purchase  7394 non-null   object 
 8   Ratings           8980 non-null   float64
 9   Date_Of_Review    8980 non-null   object 
 10  Response          8980 non-null   object 
dtypes: float64(2), object(9)
memory usage: 773.7+ KB


,Platform,Brand,Category,Product_Name,Price,Reviewer,Review,Product_Purchase,Ratings,Date_Of_Review,Response
0,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,zenheng,"delivery was quite fast, and item was on disco...",NaN,4.0,2020-02-06 23:58,no
1,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,glennoeestaquio,received item sealed properly.\r\r\nquick deli...,NaN,5.0,2020-02-06 12:44,no
2,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,shy2206,yet to try but delivery was prompt and product...,NaN,5.0,2020-02-06 10:21,no
3,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,lindamiyalin,"well wrapped, in good condition, super fast ...",NaN,5.0,2020-01-30 23:01,no
4,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,kyc385,"well packed, will buy again",NaN,5.0,2020-02-01 11:40,no


### Transformation and Initalise of data

In [7]:
# make the old csv data into a list

platform_list = data['Platform'].tolist()
brand_list = data['Brand'].tolist()
category_list = data['Category'].tolist()
product_name_list = data['Product_Name'].tolist()
price_list = data['Price'].tolist()
reviewer_list = data['Reviewer'].tolist()
review_list = data['Review'].tolist()
product_variation_list = data['Product_Purchase'].tolist()
rating_list = data['Ratings'].tolist()
date_review_list = data["Date_Of_Review"].tolist()
response_list = data["Response"].tolist()

# initialising the new columns

platform_csv=[]
brand_csv = []
category_csv = []
product_name_csv = []
prices_csv = []
reviewer_csv =[]
review_splitted_csv = []
review_csv=[]
product_variation_csv = []
rating_csv = []
date_review_csv = []
response_csv = []

# this is the length of the csv old data
print ("The length of the dataset is:", len(brand_list))

The length of the dataset is: 9002


In [12]:
#upload the appos
from word_dict import appos

print(appos)

{"aren't": 'are not', "can't": 'cannot', "couldn't": 'could not', "didn't": 'did not', "doesn't": 'does not', "don't": 'do not', "hadn't": 'had not', "hasn't": 'has not', "haven't": 'have not', "he'd": 'he would', "he'll": 'he will', "he's": 'he is', "i'd": 'I had', "i'll": 'I will', "i'm": 'I am', "isn't": 'is not', "it's": 'it is', "it'll": 'it will', "i've": 'I have', "let's": 'let us', "l'oreal": 'loreal', "mightn't": 'might not', "mustn't": 'must not', "shan't": 'shall not', "she'd": 'she would', "she'll": 'she will', "she's": 'she is', "shouldn't": 'should not', "that's": 'that is', "there's": 'there is', "they'd": 'they would', "they'll": 'they will', "they're": 'they are', "they've": 'they have', "we'd": 'we would', "we're": 'we are', "weren't": 'were not', "we've": 'we have', "what'll": 'what will', "what're": 'what are', "what's": 'what is', "what've": 'what have', "where's": 'where is', "who'd": 'who would', "who'll": 'who will', "who're": 'who are', "who's": 'who is', "who'

### Transform the Date Format

In [9]:
new_date =[]
for i in range(len(brand_list)):
    
    date_review = date_review_list[i]
    
    if platform_list[i] == 'Lazada':
        if 'days ago' in date_review or 'day ago' in date_review :
            day = date_review.replace('days ago', '').replace('day ago', '')
            date_review = datetime.today() - timedelta(days=int(day))
            date_review = date_review.strftime("%d-%b-%y")
#             print("trans",date_review)

        elif 'weeks ago' in date_review or 'week ago' in date_review:
            day = date_review.replace('weeks ago', '').replace('week ago', '')
            day = int(day)*7
            date_review = datetime.today() - timedelta(days=day)
            date_review = date_review.strftime("%d-%b-%y")
#             print("trans",date_review)

        elif 'hours ago' in date_review or 'hour ago' in date_review:
            hours = date_review.replace('hours ago', '').replace('hour ago', '')
            date_review = datetime.today() - timedelta(hours=int(hours))
            date_review = date_review.strftime("%d-%b-%y")
    
#         print("trans",date_review)
        new_date.append(date_review)
    else:
#         print(platform_list[i])
        new_date.append(date_review)
        
data["Date_Of_Review"] = new_date

new_date_review_list = data["Date_Of_Review"].tolist()

### Preprocess the Sentences

In [ ]:
import itertools
from spellchecker import SpellChecker
from nltk.tokenize import word_tokenize

spell = SpellChecker()

for i in range(len(brand_list)):
    # current_review is a string
    current_review = review_list[i]
    
    
    #print(current_review)
  
    processed_review = current_review.strip().replace("’","'")
    #print(processed_review)
    # Negation handling
    processed_review = processed_review.split()
    processed_review =[appos[w] if w in appos else w for w in processed_review]
    processed_review = " ".join(processed_review) 

    processed_review = processed_review.strip()
    # Remove all the special characters
    processed_review = re.sub(r'\W', ' ', processed_review)
    # Removing prefixed 'b'
    processed_review = re.sub(r'^b\s+', '', processed_review)
    # remove all single characters contains a white space character
    processed_review= re.sub(r'^[a-zA-Z]$', ' ', processed_review)
    # Substituting multiple spaces with single space
    processed_review = re.sub(r'\s+', ' ', processed_review, flags=re.I)


    processed = processed_review.strip()
   
    # final check if the processed_review is empty
    final = []
    if processed != "":
        tokenized = word_tokenize(processed)
        #removed additional letters eg "gooooooodddddd" to "good"
        for w in tokenized:
            processed_2 = reduce_lengthening(w)
            processed_2 = spell.correction(processed_2)
            final.append(processed_2)
        final =' '.join(final)
        print(final)
        # append into a dictionary
        platform_csv.append(platform_list[i])
        brand_csv.append(brand_list[i])
        category_csv.append(category_list[i])
        product_name_csv.append(product_name_list[i])
        prices_csv.append(price_list[i])
        reviewer_csv.append(reviewer_list[i])
        review_csv.append(current_review)
        review_splitted_csv.append(final)
        product_variation_csv.append(product_variation_list[i])
        rating_csv.append(rating_list[i])
        date_review_csv.append(new_date_review_list[i])
        response_csv.append(response_list[i])


delivery was quite fast and item was on discount too so i could not complain so far liking this product will continue to use it
received item sealed properly quick delivery too item not tested yet as for lifting purpose
yet to try but delivery was prompt and product has long expiry
well wrapped in good condition super fast delivery first time buy yet to try thanks seller excellent service
well packed will buy again
well wrap in good condition
came in good condition will start using soon thank u
soothing feel
super fast delivery received item the following day ordered during flash sale with special item coupon coins item is manufactured in jun 2019 good deal
effective product fast delivery received 1 day after order
received the products in a good condition and with fast delivery service
it is definitely cheaper than retail with the coupon I am happy with the purchase
purchase this with a promo price cheap and good thanks seller
item arrived in 3 days nicely packed good price good produ

i received lorean revitalise crystal micro essence water in good condition and with free gift thank you
received about 2 3 days after placing order together with the sunscreen used for a few days pimples seems less angry now hopefully will be able to live up to the good reviews i read online
fast delivery is definitely cheaper than retails and come with free gift can not commend much of the effect as only and day i used hope the product works well
item well received used for a few days had a good smell so far so good
product receive in very good condition just that the delivery guy simply left it at my doorstep without attempting to call or anything
received as described
best thing since sliced bread best thing since sliced bread best thing since sliced bread
received item but no free gift as advertised
fast delivery and item as described i will buy again
value for money
worth it with free sunscreen
came with the free item as advertised
saw a few advertisements about this and tried the

received within a week using it twice a week not sure if has effect already but upon using it and then you washed skin feels soft and clean shopeehaul
items received along with free lorean white perfect cleanser samples thank you
item arrived quickly as expected
well received takes about 9 days for delivery can not wait to try it tonight thank you
items well received nicely packed with freebies in bubble wrapped envelope thanks
item received with free gift however manufactured date for this product is in 2017
have not tried it yet however delivery was fast thanks
one item was manufactured in 2017 2 years old it is unacceptable
great product very effective in combating pores and blackheads yet did not dry up the cheek zones thanks for the free gift great for travel parcel delivered around one week after order
received in about a week s time meg date year 2019 comes with a free cleanser
received it really quickly thank you seller for sending it out fast the mask does not cause crackline 

received within a week there is no bubblewrap on products itself thou used a bubblewrap envelope i still think is not enough products containers are made of glass i was lucky both received in good conditions shopeehaul
receive i good condition fast delivery
have used it before very good seems to lighten my dark eye circles as well use it twice a week it does brighten dull skin g8 product
fast delivery yet to try the product giving 3 stars as seller does not respond to query at all
fast delivery and in good condition
fast delivery and good quality product thank you seller
have yet to try but excited items are well packaged and delivery was okay thank u
irresponsible and unethical seller did not send product out but lied and placed as shipped out please be on alert if you did not received your item after 10 days from the shipped out date the items are okay but the seller s unethical actions puts consumers at unnecessary risk
received and love the service yet to try the products and thank

your review has been hidden due to inappropriate content note hope may remove the coins awarded for this review
received at door gate the box did not indicate expiry date but should be not so fast expire
excited to try this out after my blue hair fades away sarah hopefully the colour would be nice
came well packed and deliver by a delivery man paid extra 4 bucks for it but its okay fast delivery will def purchase other colour n try
product received in good condition packed nicely thanks so much seller
fast delivery n nicely pack but quit disappointed that it is a really old product meg 11 2017 almost 2 years ago still sealed but box is yellow n dust x x will e chemical inside be affected
well received first time trying out this brand hope the color turn out nice
fast delivery quick respond from seller color looks nice but yet to try hope it turns out good
have yet to try but hope its nice
dirty pink was out of stock after i placed order but staff was nice to contact me and give me deta

your review has been hidden due to inappropriate content note hope may remove the coins awarded for this review
like the color not yet try if good will buy again
good deal love the rose nuance hence purchased more
vibrant colour relatively fast delivery good packaging recommended to buy
fast delivery and well packed nice colors mini lipstick is very convenient to bring out love the bag as well
received in good condition thank you
repeated orders
colour is not as rich as i expect and did not think this was a good buy since it is just ok only and will not buy again it was very mild and not as matt and rich as the advertisement promise
good product quality unfortunately when received initially there was some stains on lipstick hence requested for a replacement initial chat messages are replied promptly after item was collected back by seller replacement process was long about weeks otherwise 5 stars for seller
third time purchasing love the color so much bought during sales time and with 

good deal 10 50 with add 10 flashback have been using it for past few years but noticed recently that it has a very unpleasant choking smell when being used hope this batch will not he the smell
second time buying value for money and cheaper than in stores during sale
item received in good condition bought during flash sale which made it very value for money
fast delivery and in good condition expensive for delivery charge hope is good to use
manufacture date of black 7 2019 a bit blur manufacture date of dark iced brown 7 2018
good product
fast delivery packed in a bubble envelope price slightly cheaper than retail meg date 07 2019
really appreciate the prompt and efficient delivery will buy again during offer _
very fast delivery
repeat order fast delivery will order again in future recommended
item took some time to delivery got deal on 11 11 which is cheaper than outside
item received in good condition it took about 12 days to receive box a bit dented at the corner but lucky the gl

my first bottle almost finished hence i need a second bottles love this scents shopeehaul
box was not covered properly but luckily the bottle was not damage still a good buy
expected the rose to be bigger but hey it is ok for me personally not a big fan of the smell very perfume ish consistency is thick oil but looks nice when applied to hair the price is probably a little high because of the glass packaging delivery fast
good product quality thanks seller love it
received quite fast and bubble wrapped nicely smell good too
nice packaging hope will love the product
product delivery was very fast and looks exactly like the one i bought at retail stores
good quality items with decent price
fast delivery
was deciding between this and another product but this looks good as well
smell nice i love it
quick response from seller good products thank you for the quick delivery
order 2 bottles with free delivery prefer rose than lavender love the product
good for my frizzy hair I am currently usi

delivery took awhile lucky received yesterday so that i can use today for my photoshop tried the mascara good like the effect easy to use
super fast delivery and these are the best mascara every love love it my th time ordering
well received
good price best mascara
natural and wispy looking lashes little smudging after around 8 hours of wear on oily eyelids
cardboard backing got bent but the items themselves were fine so not too bad
very prompt delivery hope the product is effective and good
item came in bubble wrapped envelope my holy grill mascara cheaper better than all my high ends one will repurchase again again till the end of time brought it on flash sales cheaper than rtp
items were nicely packed and delivery was really quick thank you
seller packed nicely in bubble wrap envelope thank you meg in may 2019 very new will come to visit again
smudges and is unable to hold the curl but coats my lashes perfectly and does not clump
product is well received but delivery service is irre

In [ ]:
#Store into a new data frame
splitted_data = {'Platform':platform_csv ,'Brand':brand_csv, 'Category': category_csv, 'Product Name ': product_name_csv, 'Price':prices_csv ,'Reviewer':reviewer_csv,'Review':review_csv, 'Review_splitted':review_splitted_csv, 'Product Purchase':product_variation_csv,'Ratings':rating_csv,'Date Of Review':date_review_csv,'Response': response_csv }
splitted_df = pd.DataFrame.from_dict(splitted_data)
splitted_df['Date Of Review'] = pd.to_datetime(splitted_df['Date Of Review'])
splitted_df.head()
splitted_df.info()

splitted_df.to_csv('AllData_Process_Topic.csv')